In [ ]:
import os
import sys
module_path = os.path.abspath(os.path.join('/Users/theot/Documents/SEGR_0D/EGR0D/'))
if module_path not in sys.path:
    sys.path.append(module_path)

#sys.path.remove('/Users/theot/Documents/SEGR_0D/EGR0D/')
print(sys.path)

from lib_egr_260 import *
import time
from egr_equilibrate_multiproc import *
import pandas as pd

In [ ]:
if __name__ == '__main__':
    # get the start time
    st = time.time()

    config = case('CH4:1.',                     #fuel compo
                  [300],                        #tin fuel
                  [1e5],                        #pin fuel
                  'O2:1. N2:3.76',              #ox compo
                  [300],                        #tin ox
                  [1e5],                        #pin ox
                  'CO2:1.',                     #egr compo
                  [300],                        #tin egr
                  [1e5],                        #pin egr
                  [0.8,0.9,1.0,1.1,1.2],        #phi range
                  [0.0,0.1,0.3,0.5],            #egr range
                  'mole',                       #egr rate unit
                  'Aramco13.cti'                #scheme
                 )
    
    dfs=[]
    pressures = [100_000,500_000] #Pa
    for p in pressures:
        #set reservoirs thermo-state
        _, config.gas.fuel = create_reservoir(config.compo.fuel,config.scheme, 300.0, p)
        _, config.gas.ox = create_reservoir(config.compo.ox,'air.xml', 300.0, p)
        _, config.gas.egr = create_reservoir(config.compo.egr,config.scheme, 300.0, p)
        Tin = [t[i] for t in config.tin for i in len(config.tin.fuel)]
        print(Tin)
        items = [(config,phi,egr) for phi in config.phi_range for egr in config.egr_range]
        pool = mp.Pool(6)
        results = pool.starmap(compute_solutions_1D, items)
        #reactor,pdresult = compute_solutions_1D(config,1,real_egr=False)
        conc=pd.concat(results[:],axis=0)
        print(conc)
        dfs.append(conc)
        conc.to_csv('df_'+'aramco_1D_data_'+str(p/100000)+'bar'+'.csv')

    print(dfs)

    # get the end time
    et = time.time()
    # get the execution time
    elapsed_time = et - st

        
    #select the columns to plot (X,Xbis,Y)
    #dfs.to_csv('df_'+'aramco_1D_data_phi1'+'.csv')    
    

In [ ]:
readdf=[pd.read_csv('df_'+'aramco_1D_data_'+str(p/100000)+'bar'+'.csv',index_col=0) for p in pressures]
print(readdf)
readdfsc=[df.pivot_table(index='phi',columns='EGR',values='u') for df in readdf]
print(readdfsc)

title='(0D) Flame temperature vs equivalence ratio (Tin_EGR:'+str(config.res.egr.thermo.T)+'K)'
human_labels = [str(round(p/100000,1))+' bar, '+str(round(e*100,1))+"%EGR"+config.egr_unit for p in pressures for e in config.egr_range]
xlabel='Equivalence ratio'
ylabel='Tad [K]'
#readdfsc=pd.concat(readdfsc, axis = 1, keys = pressures)
# ax = [show_graphs(df,title,style='-o') for df in readdfsc]

# equilibrate_data=main(pressures)
# equilibrate_data=[df.pivot_table(index='phi',columns='EGR',values='T') for df in equilibrate_data]
# human_labels+=['Equilibrium'+str(round(p/100000,1))+' bar, '+str(round(e*100,1))+"%EGR"+config.egr_unit for p in pressures for e in config.egr_range]
# equilibrate_data=pd.concat(equilibrate_data, axis = 1)
# #equilibrate_data.plot(ax=ax, style='--',title='Temperature vs equivalence ratio',xlabel='Equivalence ratio',ylabel='T',legend=True)
#fig,ax=plt.subplots(1,1)
show_graphs(readdfsc,title,human_labels,xlabel,ylabel,style='.-',subplot=1,plot=True)# for df in readdfsc]
#plt.show()

In [ ]:
dfsc=[df.pivot_table(index='phi',columns='EGR',values='T') for df in dfs]
dfsc=pd.concat(dfsc, axis = 1, keys = pressures)

print(dfsc)
print('Execution time:', elapsed_time, 'seconds')

    

In [ ]:

#dfsccsv=pd.concat(dfs, axis = 1)
#dfsccsv.to_csv('df_no_keys_'+'aramco_1D_data'+'.csv')
title='(0D) Flame temperature vs equivalence ratio (Tin_EGR:'+str(config.res.egr.thermo.T)+'K)'
human_labels = [str(round(p/100000,1))+' bar, '+str(round(e*100,1))+"%EGR"+config.egr_unit for p in pressures for e in config.egr_range]
xlabel='Equivalence ratio'
ylabel='Tad [K]'

ax = show_graphs(dfsc,title,style='-o')

equilibrate_data=main(pressures)
equilibrate_data=[df.pivot_table(index='phi',columns='EGR',values='T') for df in equilibrate_data]
human_labels+=['Equilibrium'+str(round(p/100000,1))+' bar, '+str(round(e*100,1))+"%EGR"+config.egr_unit for p in pressures for e in config.egr_range]
equilibrate_data=pd.concat(equilibrate_data, axis = 1)
#equilibrate_data.plot(ax=ax, style='--',title='Temperature vs equivalence ratio',xlabel='Equivalence ratio',ylabel='T',legend=True)

show_graphs(equilibrate_data,title,human_labels,xlabel,ylabel,style='--',ax=ax,plot=True)
